This notebook is dedicated to improving, or building a brand new summary section.

In [2]:
with open("job_desc.md") as f:
    description = f.read()
with open("resume.md") as f:
    resume = f.read()
title = 'django developer'

In [1]:
from util import chunk_markdown, cut_sections

In [3]:
sections = cut_sections(resume)

In [5]:
summary = sections['summary']

Two different independent chains will be built to improve summary sections:
1. Take existing summary and tailor it for job description.
2. Use the revised job experience section to extract demonstrations of required skills (from job description) as statements, statistics, or stories. The list of demonstrations will then undergo elimination, condensation, and ordering to build a summary section.

Summary is built by default, in the event that the input document doesn't include a summary.

## Improve existing summary section

This assumes that the writer provided sufficient text in the existing summary and presented strong enough arguments.
 
The role hallucination serves has not been verified. A certain degree of "creativity" is desired to truly reframe the existing content to retain appeal while conveying confidence and competence. However, the algorithm stating facts that aren't true to fit the job description is not desired.

In [18]:

from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain


def improve_summary_chain() -> SequentialChain:
    llm = ChatOpenAI(temperature=.2, model_name="gpt-3.5-turbo")

    appeal_prompt = PromptTemplate.from_template("""
    You will be given a resume section. You will also be given a job description.
    Improve the given resume section to be more appealing to a recruiter looking to fill the given job description.
    The goal is to convey confidence and competence.
    Return only the improved summary, with no header.
    
    Resume Section:
    {section}
    
    Job Description:
    {desc}
    """)
    appeal_chain = LLMChain(prompt=appeal_prompt, llm=llm, output_key="improved")

    summ_prompt = PromptTemplate.from_template("""
    Condense the following resume summary section using {format}.
    The summary should remain appealing to a recruiter and be less than 260 characters.
    
    Resume section:
    {improved}
    """,
                                               partial_variables={'format': 'a mix of bullets and 1 short paragraph'})
    summ_chain = LLMChain(prompt=summ_prompt, llm=llm, output_key="summarized")

    return SequentialChain(
        chains=[appeal_chain, summ_chain],
        input_variables=["section", "desc"],
        output_variables=["summarized"],
    )

In [19]:
improve_summary_chain = improve_summary_chain()
improve_summary_chain({'section': summary, 'desc': description, 'format': '3 bullets and 1 sentence'})


{'section': {'section_name': 'resume summary',
  'result': "# Summary\n- Over 6 combined years of experience maintaining complex systems. Over 10 years of Python programming experience. Developed clean and maintainable code for the last 4 years.\n- Built projects involving Python, Rust, React, django/flask, and Angular for user interfaces (UI's), websites, automated trading systems, control system libraries, bioinformatic tools, and API wrappers.\n- Experience with git/GitHub, AWS, Docker, HTML/CSS, RUST API's, and extensive testing."},
 'desc': 'Vendelux (https://vendelux.com/) is the system of record for event marketing. Our software platform provides proprietary data that helps high-growth companies find the highest ROI events, conferences and trade shows to attend and sponsor. We have built a product that customers describe as an “event marketer’s dream” and are backed with venture capital funding and top-tier clients.\n\nWe are now hiring more engineers to help us build the best p